In [1]:
from pyspark import SparkConf
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql import functions as f

In [2]:
!hdfs dfsadmin -safemode leave

Safe mode is OFF


In [2]:
!hdfs dfs -rm -r /ml-latest-small
!hdfs dfs -put ml-latest-small /

Deleted /ml-latest-small


## 1. Spark Application


Два Executor

In [3]:
conf = (
    SparkConf()
    .set("spark.executor.instances", "2")
    .set("spark.executor.cores", "2")
    .set("spark.executor.memory", "512m")
)

In [4]:
spark = (
    SparkSession
    .builder
    .master(master="yarn")
    .config(conf=conf)
    .appName("sibagatulin_spark")
    .getOrCreate()
)

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/12/12 19:44:40 WARN Client: Neither spark.yarn.jars nor spark.yarn.archive is set, falling back to uploading libraries under SPARK_HOME.


In [5]:
spark

YARN application:

<img src="img/task_1 spark app in yarn.png" alt="image" width="700" height="auto">

Spark UI:

<img src="img/task_1 spark ui.png" alt="image" width="700" height="auto">

In [8]:
%%time
df = spark.range(int(1e6))

CPU times: user 2.4 ms, sys: 414 µs, total: 2.81 ms
Wall time: 63.8 ms


In [14]:
%%time
df.take(10)

CPU times: user 7.61 ms, sys: 0 ns, total: 7.61 ms
Wall time: 132 ms


[Row(id=0),
 Row(id=1),
 Row(id=2),
 Row(id=3),
 Row(id=4),
 Row(id=5),
 Row(id=6),
 Row(id=7),
 Row(id=8),
 Row(id=9)]

In [7]:
%%time

tags_schema = StructType(fields=[
    StructField("userId", IntegerType()),
    StructField("movieId", IntegerType()),
    StructField("tag", StringType()),
    StructField("timestamp", LongType()),
])

tags_df = (
    spark
    .read
    .format("csv")
    .option("header", "True")
    .schema(tags_schema)
    .load("/ml-latest-small/tags.csv")
)

CPU times: user 8.57 ms, sys: 0 ns, total: 8.57 ms
Wall time: 1.49 s


In [8]:
%%time
tags_df.show(10)

ERROR:root:Exception while sending command.                                     
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/py4j/clientserver.py", line 516, in send_command
    raise Py4JNetworkError("Answer from Java side is empty")
py4j.protocol.Py4JNetworkError: Answer from Java side is empty

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/usr/local/lib/python3.10/dist-packages/py4j/clientserver.py", line 539, in send_command
    raise Py4JNetworkError(
py4j.protocol.Py4JNetworkError: Error while sending or receiving
ERROR:root:Exception while sending command.
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/py4j/clientserver.py", line 516, in send_command
    raise Py4JNetworkError("Answer from Java s

Py4JError: An error occurred while calling o46.showString

In [ ]:
ratings_schema = StructType(fields=[
    StructField("userId", IntegerType()),
    StructField("movieId", IntegerType()),
    StructField("rating", DoubleType()),
    StructField("timestamp", LongType()),
])

ratings_df = (
    spark
    .read
    .format("csv")
    .option("header", "True")
    .schema(ratings_schema)
    .load("/ml-latest-small/ratings.csv")
)

In [ ]:
ratings_df.take(5)